## legistar without videos available

'https://phila.legistar.com/Legislation.aspx'

'https://columbus.legistar.com/Legislation.aspx'

'https://phoenix.legistar.com/Legislation.aspx'

## legistar with videos available

'https://seattle.legistar.com/Legislation.aspx' ## rescrape Seattle Channel from calendar landing page (should be quick)

'https://newark.legistar.com/Legislation.aspx' ## rescrape each page (as in Durham)

'https://durhamcounty.legistar.com/Legislation.aspx' ## see above (in progress)

'https://alexandria.legistar.com/Legislation.aspx' ##  rescrape each page (as in Durham)

## potentially new sources

https://denver.legistar.com/Calendar.aspx

https://jaxcityc.legistar.com/Calendar.aspx

https://fortworthgov.legistar.com/Calendar.aspx

In [93]:
import pandas as pd 

In [94]:
df = pd.read_csv('../data/jacksonville/jaxcityc__policy_info.csv', index_col=0)

In [97]:
df.shape

(31318, 16)

In [103]:
df[['key', 'video_url']].loc[lambda df: df['video_url'].notnull()].drop_duplicates()

,key,video_url
File #,,
2024-0098-E,MeetingDetail.aspx?ID=1110896&GUID=44CDCE20-00...,https://jaxcityc.granicus.com/player/clip/5738...
2024-0061-E,MeetingDetail.aspx?ID=1110893&GUID=A10CF4BE-28...,https://jaxcityc.granicus.com/player/clip/5592...
2024-0230-E,MeetingDetail.aspx?ID=1110897&GUID=CCD8CADE-FA...,https://jaxcityc.granicus.com/player/clip/5768...
2024-0067-A,MeetingDetail.aspx?ID=1110891&GUID=A70C363A-E8...,https://jaxcityc.granicus.com/player/clip/5552...
2023-0752-E,MeetingDetail.aspx?ID=1110881&GUID=3AB25062-6A...,https://jaxcityc.granicus.com/player/clip/5444...
...,...,...
2018-0712-W,MeetingDetail.aspx?ID=713167&GUID=BB954877-B9B...,https://jaxcityc.granicus.com/player/clip/908?...
No records to display.,MeetingDetail.aspx?ID=713566&GUID=9B1E1FC8-085...,https://jaxcityc.granicus.com/player/clip/895?...
No records to display.,MeetingDetail.aspx?ID=712734&GUID=2EE84E50-199...,https://jaxcityc.granicus.com/player/clip/876?...


In [105]:
df['video_url'].drop_duplicates().iloc[0]

'https://jaxcityc.granicus.com/player/clip/5738?view_id=1&redirect=true'

In [106]:
import os
import urllib

In [110]:
t = urllib.parse.quote(df['video_url'].drop_duplicates().iloc[0] , safe='')

In [112]:
urllib.parse.unquote(t)

'https://jaxcityc.granicus.com/player/clip/5738?view_id=1&redirect=true'

In [113]:
t

'https%3A%2F%2Fjaxcityc.granicus.com%2Fplayer%2Fclip%2F5738%3Fview_id%3D1%26redirect%3Dtrue'

# Denver, Fortworth, Jacksonville, Durham

In [315]:
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeoutError

playwright = await async_playwright().start()
# browser = await playwright.firefox.launch(headless=False)
browser = await playwright.chromium.launch(headless=False)
context = await browser.new_context()
page = await context.new_page()

In [316]:
import logging
async def download_with_progress(page, download_sel, download_path):
    def sizeof_fmt(num, suffix="B"):
        for unit in ("", "Ki", "Mi", "Gi", "Ti", "Pi", "Ei", "Zi"):
            if abs(num) < 1024.0:
                return f"{num:3.1f}{unit}{suffix}"
            num /= 1024.0
        return f"{num:.1f}Yi{suffix}"

    # Start the download
    async with page.expect_download() as download_info:
        # Perform the action that initiates download
        await page.locator(download_sel).click()
        download = await download_info.value

    # Wait for the download to start
    download_path_temp = await download.path()
    while not os.path.exists(download_path_temp):
        print("Waiting for file to appear on disk...")
        await asyncio.sleep(0.1)  # Check every 100ms

    print("File detected, starting to track progress...")

    # Track the download progress
    while not download.is_finished():
        if os.path.exists(download_path_temp):
            current_size = os.path.getsize(download_path_temp)
            print(f"Download Progress: {sizeof_fmt(current_size)}")
        else:
            print("File no longer accessible.")
        await asyncio.sleep(1)  # Update every second

    await download.save_as(download_path)
    print("Download complete and saved to: " + download_path)

In [317]:
video_url = 'https://denver.granicus.com/player/clip/16130?view_id=180&redirect=true'
video_url = 'https://fortworthgov.granicus.com/player/clip/5506?view_id=5&redirect=true'
# video_url = "https://jaxcityc.granicus.com/player/clip/3584?view_id=1&redirect=true"
# video_url = 'https://jaxcityc.granicus.com/player/clip/5266?view_id=1&redirect=true'
try: 
    await page.goto(video_url, timeout=10_000)
    await page.wait_for_selector(video_tag_sel)
except PlaywrightTimeoutError as e:
    pass

In [318]:
video_tag_sel = 'video'

In [319]:
video_tag_sel = '#video > div.flowplayer.fp-full.fp-edgy.fp-mute.fp-default-playlist.is-ready.is-long.is-paused.is-mouseout > div.fp-player > video > source'

In [320]:
from bs4 import BeautifulSoup

html = await page.content()
soup = BeautifulSoup(html)

In [321]:
import ffmpeg
from ffmpeg.errors import FFmpegError

def download_video(url, output_file):
    try:
        print(f"starting download: {output_file}")
        # Stream input from URL
        input_stream = ffmpeg.input(url)
        # Copy video and audio streams without re-encoding
        output_stream = ffmpeg.output(input_stream, output_file, codec='copy')
        # Run and overwrite output file if exists
        ffmpeg.run(output_stream, overwrite_output=True, quiet=True)
        print(f"Download complete: {output_file}")
    except FFmpegError as e:
        print("An error occurred:", e)

In [323]:
download_sel = '#navigation-display-download > span'
exists_download_button = await page.query_selector(download_sel)
await page.locator(download_sel).click()
video_sel = '#download-options > a > span > i.fa.fa-file-video-o.fa-stack-1x.fa-inverse'
exists_video_download_button = await page.query_selector(video_sel)

In [324]:
await download_with_progress(page, video_sel, 'test.mp4')

File detected, starting to track progress...


AttributeError: 'Download' object has no attribute 'is_finished'

In [259]:
if exists_download_button:
    async with page.expect_download() as download_info:
        # Perform the action that initiates download
        await page.locator(download_sel).click()
        audio_sel = '#download-options > a:nth-child(2) > span > i.fa.fa-file-audio-o.fa-stack-1x.fa-inverse'
        exists_audio_download_button = await page.query_selector(audio_sel)
        if exists_audio_download_button:
            await page.locator(audio_sel).click()
    
        video_sel = '#download-options > a > span > i.fa.fa-file-video-o.fa-stack-1x.fa-inverse'
        exists_video_download_button = await page.query_selector(video_sel)
        if exists_video_download_button:
            await page.locator(video_sel).click()
                
    download = await download_info.value
    await download.save_as(f"/Users/spangher/Downloads/test-{download.suggested_filename}")

## download the stream by hand
else:
    print('download button doesn\'t exist')
    video_tag_sel = '#video > div.flowplayer.fp-full.fp-edgy.fp-mute.fp-default-playlist.is-ready.is-long.is-paused.is-mouseout > div.fp-player > video > source'
    video_tag = soup.select(video_tag_sel)
    if len(video_tag ) > 0:
        video_tag = video_tag[0]
    playlist_url = video_tag.attrs['src']
    download_video(playlist_url, 'test.mp4')

Error: Download.save_as: canceled

In [147]:
video_tag_sel = '#video > div.flowplayer.fp-full.fp-edgy.fp-mute.fp-default-playlist.is-ready.is-long.is-paused.is-mouseout > div.fp-player > video > source'
video_tag = soup.select(video_tag_sel)
if len(video_tag ) > 0:
    video_tag = video_tag[0]
playlist_url = video_tag.attrs['src']


In [148]:
playlist_url

'https://archive-stream.granicus.com/OnDemand/_definst_/mp4:archive/jaxcityc/jaxcityc_84383b12-344f-4c1f-906e-0a8350747a02.mp4/playlist.m3u8'

# Seattle

In [204]:
import wget
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeoutError

playwright = await async_playwright().start()
browser = await playwright.firefox.launch(headless=False)
context = await browser.new_context()
page = await context.new_page()

In [205]:
video_url = 'https://www.seattlechannel.org/videos?videoid=x156928&Mode2=Video'
video_url = 'https://www.seattlechannel.org/videos?videoid=x150763&amp;Mode2=Video'
# video_url = 'https://www.seattlechannel.org/videos?videoid=x156889&amp;Mode2=Video'
try:
    await page.goto(video_url)
except PlaywrightTimeoutError as e:
    pass

In [206]:
await page.locator('#vidPlayer').click()
await page.hover('#vidPlayer')
await page.locator('''
    #vidPlayer > 
        div.jw-wrapper.jw-reset > 
        div.jw-controls.jw-reset > 
        div.jw-controlbar.jw-reset > 
        div.jw-reset.jw-button-container > 
        div:nth-child(13) > 
        div.jw-icon.jw-button-image.jw-button-color.jw-reset
''').click()

In [207]:
page_url = page.url

In [208]:
page_url

'https://video.seattle.gov/media/council/assets_092023_2702335V.mp4'

In [121]:
import wget

In [209]:
filename = wget.download(page_url, out='seattle-test-output.mp4')

100% [......................................................................] 639562753 / 639562753

In [214]:
! mediainfo --Inform="Video;%Codec%" seattle-test-output.mp4

# Newark, Alexandria

In [201]:
video_url = 'https://newark.granicus.com/MediaPlayer.php?view_id=2&clip_id=622'
video_url = "https://alexandria.granicus.com/MediaPlayer.php?view_id=53&clip_id=6129"

In [203]:
await page.goto(video_url)

download_sel = '#download-menu'
await page.locator(download_sel).click()

async with page.expect_download() as download_info:
    # Perform the action that initiates download
    download_sel = '#downloadVideo'
    await page.locator(download_sel).click()

download = await download_info.value
await download.save_as(f"/Users/spangher/Downloads/test-{download.suggested_filename}")

Exception: Page.goto: Connection closed while reading from the driver

In [45]:
### parse HTML panel

In [40]:
html = await page.content()

In [41]:
soup = BeautifulSoup(html)

In [ ]:
soup.find(attrs={'id': 'content-pane'}).find_all('a')

In [169]:
playlist_url = 'https://archive-stream.granicus.com/OnDemand/_definst_/mp4:archive/jaxcityc/jaxcityc_96443eb6-12b2-11ef-b231-0050569183fa.mp4/playlist.m3u8'

In [182]:
! ffmpeg -y -i $playlist_url -c copy -t 30 output.mp4

ffmpeg version 6.1.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with clang version 16.0.6
  configuration: --prefix=/Users/runner/miniforge3/conda-bld/ffmpeg_1712656609600/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pl --cc=arm64-apple-darwin20.0.0-clang --cxx=arm64-apple-darwin20.0.0-clang++ --nm=arm64-apple-darwin20.0.0-nm --ar=arm64-apple-darwin20.0.0-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-libopenh264 --enable-libdav1d --enable-cross-compile --arch=arm64 --target-os=darwin --cross-prefix=arm64-apple-darwin20.0.0- --host-cc=/Users/runner/miniforge3/conda-bld/ffmpeg_1712656609600/_build_env/bin/x86_64-apple-darwin13.4.0-clang --enable-neon --enable-gnutls --enable-libmp3lame --enable-libvpx --enable-libass -

In [190]:
! ffprobe -v error -show_entries format=duration -of default=noprint_wrappers=1:nokey=1 output-audio.mp3

30.048000


In [195]:
import subprocess 



In [200]:
float(output)

30.016

In [186]:
! ffmpeg -y -i output.mp4 -vn -q:a 0 -map a output-audio.mp3

ffmpeg version 6.1.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with clang version 16.0.6
  configuration: --prefix=/Users/runner/miniforge3/conda-bld/ffmpeg_1712656609600/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pl --cc=arm64-apple-darwin20.0.0-clang --cxx=arm64-apple-darwin20.0.0-clang++ --nm=arm64-apple-darwin20.0.0-nm --ar=arm64-apple-darwin20.0.0-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-libopenh264 --enable-libdav1d --enable-cross-compile --arch=arm64 --target-os=darwin --cross-prefix=arm64-apple-darwin20.0.0- --host-cc=/Users/runner/miniforge3/conda-bld/ffmpeg_1712656609600/_build_env/bin/x86_64-apple-darwin13.4.0-clang --enable-neon --enable-gnutls --enable-libmp3lame --enable-libvpx --enable-libass -

In [184]:
! open output.mp4

In [188]:
! open output-audio.mp3